# Webscrape college applications
## Step One: Prepare Directory Information

In [ ]:
import sys
# log_file = open('log_file.txt', 'w')
# sys.stdout = log_file
# print('End of cell')

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [ ]:
# Load directory data.
IPEDSfile = pd.read_stata('../../statadata/IPEDSDirInfo02to16smlr.dta', preserve_dtypes=False)
# Reduce to 2 and 4yr institutions.
IPEDSfile['filter'] = np.where((IPEDSfile['sector']=='Public, 4-year or above') | 
                               (IPEDSfile['sector']=='Public, 2-year') |
                               (IPEDSfile['sector']=='Private not-for-profit, 4-year or above') |
                               (IPEDSfile['sector']=='Private not-for-profit, 2-year'), 1, 0)
IPEDSfile = IPEDSfile[IPEDSfile['filter']==1]

In [ ]:
# Remove www. prefix from webaddress.
IPEDSfile['rootdom'] = IPEDSfile.loc[:, 'webaddr'].replace(regex=True, to_replace='www.', value='')
# Remove miscellaneous slashes & other from webaddress.
IPEDSfile['rootdom'] = IPEDSfile.loc[:, 'rootdom'].replace(regex=True, to_replace=r'/', value='')
IPEDSfile['rootdom'] = IPEDSfile.loc[:, 'rootdom'].replace(regex=True, to_replace=r'HTTPS:', value='')

In [ ]:
IPEDSfile[['unitid','instnm','rootdom','filter','sector','iclevel']].head(n=15)

In [ ]:
IPEDSfile.describe()

In [ ]:
# Create list of domains to scrape
weblist = IPEDSfile['rootdom']

## Step Two: Prepare Browser Object

In [ ]:
import webbrowser
from time import sleep
from selenium import webdriver
import os
import requests
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
browser = webdriver.Firefox()
print('Loaded Browser Here')
# browser = webdriver.Chrome()

In [ ]:
# browser.get(r'https://www.google.com/search?as_q=early+decision&as_epq=&as_sitesearch=.edu&as_filetype=pdf')
# print('send browser to search')

# Step Three: Scrape For Each School

In [ ]:
error_list = []
for school in weblist:
    print('STARTING SCHOOL', str(school).upper())
    searchstr = ''.join((r'https://www.google.com/search?as_q=early+decision&as_epq=&as_sitesearch=', 
                        school, '&as_filetype=pdf'))
    browser.get(searchstr)
    print('Search string is ', searchstr)
    results = browser.find_elements_by_css_selector('h3 > a')
    if len(results) == 0:
        try:
            check_captcha = browser.find_element_by_partial_link_text('Why did this happen')
            print('At school, ', school.lower(), 'Google responded with CAPTCHA - will exit program.', end='\n\n')
            error_list.append(''.join((school, '-', '.', '-EncounteredCAPTCHA-', '.')))
            sys.exit()
        else:
            print('No results for school ', school.lower(), end='\n\n')
            error_list.append(''.join((school, '-', '.', '-NoResults-', '.')))
    for i in range (len(results)):
        filelink = results[i].get_attribute('href')
        try:
            pdf = requests.get(filelink)
            fname = ''.join((school,str(i),'.pdf'))
            print('Filelink = ' + filelink)
            print('Saved as : ' + fname)
            print('Header info :' + str(pdf.headers), end='\n\n')
            open(''.join((r'pdfs/', fname)), 'wb').write(pdf.content)
        except ConnectionError:
            print('There was a ConnectionError on the {}th iteration at : {}'.format(str(i), school.lower()))
            error_list.append(''.join((school, '-', str(i), '-ConnectionError-', filelink)))
        except:
            print('There was an UnspecifiedError on the {}th iteration at : {}'.format(str(i), school))
            error_list.append(''.join((school, '-', str(i), '-UnspecifiedError-', filelink)))